In [9]:
# Imports
import os
from pkg_resources import resource_filename
import numpy as np
from astropy.io import fits

# ASTR 257
from astr257 import img_utils

# Personal utils
from utils import *

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams
rcParams["font.family"] = "serif"
rcParams["font.serif"] = "Times New Roman"
%matplotlib inline
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = [r'\usepackage{amsmath} \usepackage{bm} \usepackage{physics}']
%config InlineBackend.figure_format = 'retina' # For high quality figures
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data_folder = "../data-2019-09-25-shane-joey-alex-arcelia"

In [11]:
OVERWRITE = True

## Load the calibration masters

In [12]:
biases_combined = fits.getdata(os.path.join(data_folder, 'masters/biases_combined.fits'))
darks_master    = fits.getdata(os.path.join(data_folder, 'masters/darks_master.fits'))
flats_master    = fits.getdata(os.path.join(data_folder, 'masters/flats_master_normalized.fits')) # Flats are norm'd

# Download the data:
https://mtham.ucolick.org/data/2019-09/23/nickel/Andrew.Skemer/

## Load and median combine the science images

In [13]:
ugc11876_combined, header_ugc11876 = crawl_and_median(data_folder, 'ugc11876')

## Throwout the part of the image that the slit is covering

In [14]:
ugc11876_combined = ugc11876_combined[:, 90:348]

In [15]:
ugc11876_combined.shape

(2725, 258)

## Calibrate the science image


In [16]:
foo_factor = 1.1 * np.abs(np.min(flats_master))

In [17]:
ugc11876_master = ugc11876_combined - darks_master - biases_combined / (flats_master + foo_factor)

## Save the calibrated image

In [18]:
# Save
try:
    fits.writeto(os.path.join(data_folder, 'masters/ugc11876_master.fits'), 
                 ugc11876_master, header=header_ugc11876, overwrite=OVERWRITE)
except:
    print('Warning: Did not overwrite file.')